In [1]:
# Import Modules
import pandas as pd
import scipy.stats as stats
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Read csv file into dataframe
df = pd.read_csv('paysim.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
step              int64
type              object
amount            float64
nameOrig          object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest          object
oldbalanceDest    float64
newbalanceDest    float64
isFraud           int64
isFlaggedFraud    int64
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [3]:
# Pearson's Chi-squared test of categorical variable "Type" for category "isFraud" 
# H0 : P(isFraud == 1) = P(isFraud == 0) or the variable is independent
# Ha : P(isFraud == 1) ≠ P(isFraud == 0) or the varibale is dependent 

type_split = pd.crosstab(df['type'] , df['isFraud'])
type_split

isFraud,0,1
type,,
CASH_IN,1399284,0
CASH_OUT,2233384,4116
DEBIT,41432,0
PAYMENT,2151495,0
TRANSFER,528812,4097


In [13]:
# P-Value of 0.0 would reject the null hypothesis at any level, or the test result shows dependence.
stats.chi2_contingency(type_split)

(22082.53571319108, 0.0, 4, array([[1.39747778e+06, 1.80622440e+03],
        [2.23461179e+06, 2.88821075e+03],
        [4.13785187e+04, 5.34812728e+01],
        [2.14871781e+06, 2.77719374e+03],
        [5.32221110e+05, 6.87889834e+02]]))

In [5]:
# Split the dataframe by "isfraud" category
fraud_yes = df[df.isFraud == 1]
fraud_no = df[df.isFraud == 0]

In [6]:
fraud_no.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,"6,354,407.00","6,354,407.00","6,354,407.00","6,354,407.00","6,354,407.00","6,354,407.00","6,354,407.00","6,354,407.00"
mean,243.24,"178,197.04","832,828.71","855,970.23","1,101,420.87","1,224,925.68",0.00,0.00
std,142.14,"596,236.98","2,887,144.03","2,924,986.96","3,399,201.79","3,673,815.71",0.00,0.00
min,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00
25%,156.00,"13,368.40",0.00,0.00,0.00,0.00,0.00,0.00
50%,239.00,"74,684.72","14,069.00",0.00,"133,311.80","214,881.70",0.00,0.00
75%,334.00,"208,364.76","106,969.50","144,730.74","944,144.58","1,111,975.34",0.00,0.00
max,718.00,"92,445,516.64","43,818,855.30","43,686,616.33","356,015,889.35","356,179,278.92",0.00,0.00


In [7]:
fraud_yes.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,"8,213.00","8,213.00","8,213.00","8,213.00","8,213.00","8,213.00","8,213.00","8,213.00"
mean,368.41,"1,467,967.30","1,649,667.61","192,392.63","544,249.62","1,279,707.62",1.00,0.00
std,216.39,"2,404,252.95","3,547,719.44","1,965,666.46","3,336,420.95","3,908,816.53",0.00,0.04
min,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
25%,181.00,"127,091.33","125,822.44",0.00,0.00,0.00,1.00,0.00
50%,367.00,"441,423.44","438,983.45",0.00,0.00,"4,676.42",1.00,0.00
75%,558.00,"1,517,771.48","1,517,771.48",0.00,"147,828.66","1,058,725.22",1.00,0.00
max,743.00,"10,000,000.00","59,585,040.37","49,585,040.37","236,230,516.82","236,726,494.66",1.00,1.00


In [8]:
# Manually calculated t-stat, for sanity check on t-test for obtaining p-value of difference of means test on "Amount"
# H0 : Amount(isFraud == 1) = Amount(isFraud == 0) 
# Ha : Amount(isFraud == 1) ≠ Amount(isFraud == 0) 

nonfraud_amount = fraud_no.amount
fraud_amount = fraud_yes.amount

n1 = len(nonfraud_amount)
n0 = len(fraud_amount)
print("The number of nonfraud amounts is " + str(n1))
print("The number of fraud amounts is " + str(n0))

s1 = np.std(nonfraud_amount, ddof=1)
s0 = np.std(fraud_amount, ddof=1)
print("The std amount nonfraud is " + str(s1))
print("The std amount with fraud is " + str(s0))

x1 = np.mean(nonfraud_amount)
x0 = np.mean(fraud_amount)
print("The mean amount nonfraud is " + str(x1))
print("The mean amount with fraud is " + str(x0))

sp = np.sqrt(((n0 -1) * s0 ** 2 + (n1 -1) * s1 ** 2) / (n0 + n1 -2))
print("The calculated stdev_diff is " + str(sp))

print("The mean fraud minus nonfraud amount is " + str(np.mean(fraud_amount)-np.mean(nonfraud_amount)))

t_stat = (x0 - x1) / (sp * np.sqrt(((1/n0) + (1/n1))))
print("The calculated t-stat is " + str(t_stat))            


The number of nonfraud amounts is 6354407
The number of fraud amounts is 8213
The std amount nonfraud is 596236.9813471739
The std amount with fraud is 2404252.9472401612
The mean amount nonfraud is 178197.04172739814
The mean amount with fraud is 1467967.299140387
The calculated stdev_diff is 602079.9804398556
The mean fraud minus nonfraud amount is 1289770.257412989
The calculated t-stat is 194.01200466038233


In [9]:
# P-Value of 0.0 would reject the null hypothesis at any level
# Or the test result shows the mean Amount is not the same for "isFraud" categories 
from scipy.stats import ttest_ind
ttest_ind(fraud_amount, nonfraud_amount)

Ttest_indResult(statistic=194.01200466037974, pvalue=0.0)